In [1]:
from random import randint
import numpy as np
from numpy import ndarray as nd
import json
import os

# Set random fishermen strengths and random number of trees
strengths = [randint(1,3) for i in range(3)]
ntrees = randint(1,3)

In [55]:
### HELPER: CALCULATE REWARDS
# 'strengths' : array of (integer) player strengths
# 'trees' : integer
# 'action': array of booleans that is True wherever an agent went to fish

# Returns 'fish_fished': integer describing payoff for current world
def get_rewards(strengths, trees, actions):
    n = range(len(strengths)) # Used for iteration
    
    # Calculate the number of trees cleared
    trees_cleared = sum([strengths[i] for i in n if not actions[i]])
    
    # Calculate the payoff (which is 0 if the trees are still blocking the road)
    fish_fished = sum([strengths[i] for i in n if actions[i]]) if trees_cleared >= trees else 0
    
    return fish_fished

### HELPER: COMPUTE PAYOFF MATRIX
# 'strengths' : array of (integer) player strengths
# 'trees' : integer

# Returns 'payoff_matrix': summary of payoffs for all possible actions
def get_payoff_matrix(strengths, trees):
    payoff_matrix = np.empty([2]*len(strengths)) # creates n-dimensional array
    action_space = [0, 1] # possible actions
    
    # Calculate payoffs for each possible action
    for x in action_space:
        for y in action_space:
            for z in action_space:
                payoff_matrix[x, y, z] = get_rewards(strengths, trees, [x, y, z])
                
    return payoff_matrix

# HELPER: FIND ALL SOLUTIONS IN PAYOFF MATRIX
# 'payoff_matrix': Output of get_payoff_matrix

# Returns 'solutions': a matrix of booleans that is 'true' if the given set of actions is an optimum solution
def get_solutions(p):
    return np.logical_and(p == np.amax(p), np.greater(p, 0))

# Counts the number of equilibria in a given payoff matrix
def n_solutions(strengths, trees):
    payoff = get_payoff_matrix(strengths, trees)
    solutions = get_solutions(payoff)
    
    return np.sum(solutions)

def max_payoff(s,t):
    return np.amax(get_payoff_matrix(s,t))

def get_optimal_actions(strengths,trees):
    payoff = get_payoff_matrix(strengths, trees)
    solutions = get_solutions(payoff)
    solution_indxs = np.where(solutions.flatten())
    action_space = [[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]] #based on actions in for loop
    return [action_space[optimal_action] for optimal_action in solution_indxs[0]]

In [56]:
# Iterate over all possible actions
all_strengths = [[x, y, z] for x in range(1,4) for y in range(1,4) for z in range(1,4)]
all_scenarios = [(s, t, n_solutions(s, t), max_payoff(s,t)) for s in all_strengths for t in range(1,4)]
n_solns = [n_solutions(s[0], s[1]) for s in all_scenarios]

possible_solns = np.unique(n_solns)
scenario_groups = [[s[0] for s in zip(all_scenarios, n_solns) if s[1] == u] for u in possible_solns]
scenarios = dict(zip(possible_solns, scenario_groups))

keys = ['strengths', 'trees', 'n_solutions', 'max_payoff']

scenario_info = [[dict(zip(keys, trial)) for trial in group] for group in scenario_groups]

#scenario_info = [dict(zip(keys, trial)) for group in scenario_groups for trial in group]

In [59]:
all_2_trials = [s for s in range(0,len(all_scenarios)) if all_scenarios[s][2] == 2 ]
strengths2 = [all_scenarios[s][0] for s in all_2_trials]
trees2 = [all_scenarios[s][1] for s in all_2_trials]
actions2 = [get_optimal_actions(strength, trees) for strength,trees in zip(strengths2, trees2)]

all_3_trials = [s for s in range(0,len(all_scenarios)) if all_scenarios[s][2] == 3 ]
strengths3 = [all_scenarios[s][0] for s in all_3_trials]
trees3 = [all_scenarios[s][1] for s in all_3_trials]
actions3 = [get_optimal_actions(strength, trees) for strength,trees in zip(strengths3, trees3)]

all_1_trials = [s for s in range(0,len(all_scenarios)) if all_scenarios[s][2] == 1 ]
strengths1 = [all_scenarios[s][0] for s in all_1_trials]
trees1 = [all_scenarios[s][1] for s in all_1_trials]
actions1 = [get_optimal_actions(strength, trees) for strength,trees in zip(strengths1, trees1)]

actions = get_optimal_actions([1,1,2],2)
print strengths2
print trees2
print 'actions'
print actions2

print strengths3
print trees3
print 'actions'
print actions3

print strengths1
print trees1
print 'actions'
print actions1

[[1, 1, 2], [1, 1, 2], [1, 1, 2], [1, 1, 3], [1, 2, 1], [1, 2, 1], [1, 2, 1], [1, 2, 2], [1, 2, 2], [1, 2, 3], [1, 3, 1], [1, 3, 2], [1, 3, 3], [1, 3, 3], [2, 1, 1], [2, 1, 1], [2, 1, 1], [2, 1, 2], [2, 1, 2], [2, 1, 3], [2, 2, 1], [2, 2, 1], [2, 2, 3], [2, 2, 3], [2, 3, 1], [2, 3, 2], [2, 3, 2], [2, 3, 3], [3, 1, 1], [3, 1, 2], [3, 1, 3], [3, 1, 3], [3, 2, 1], [3, 2, 2], [3, 2, 2], [3, 2, 3], [3, 3, 1], [3, 3, 1], [3, 3, 2]]
[1, 2, 3, 1, 1, 2, 3, 2, 3, 3, 1, 3, 2, 3, 1, 2, 3, 2, 3, 3, 2, 3, 1, 2, 3, 1, 2, 3, 1, 3, 2, 3, 3, 1, 2, 3, 2, 3, 3]
actions
[[[0, 1, 1], [1, 0, 1]], [[0, 0, 1], [1, 1, 0]], [[0, 1, 0], [1, 0, 0]], [[0, 1, 1], [1, 0, 1]], [[0, 1, 1], [1, 1, 0]], [[0, 1, 0], [1, 0, 1]], [[0, 0, 1], [1, 0, 0]], [[1, 0, 1], [1, 1, 0]], [[0, 0, 1], [0, 1, 0]], [[0, 0, 1], [1, 1, 0]], [[0, 1, 1], [1, 1, 0]], [[0, 1, 0], [1, 0, 1]], [[1, 0, 1], [1, 1, 0]], [[1, 0, 1], [1, 1, 0]], [[1, 0, 1], [1, 1, 0]], [[0, 1, 1], [1, 0, 0]], [[0, 0, 1], [0, 1, 0]], [[0, 1, 1], [1, 1, 0]], [[0, 0, 1],

In [15]:
with open('scenarios.json', 'w') as f:
     json.dump(scenario_info, f)

In [6]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

random_subset = [np.random.choice(group, size = 8, replace = False) for group in scenario_info]
random_subset = np.vstack(random_subset).reshape((-1,),order='F')
random_subset = split_list(random_subset, 8)
random_subset = [np.random.permutation(block) for block in random_subset]
random_subset = np.vstack(ranom_subset).reshape(-1,)



ValueError: Cannot take a larger sample than population when 'replace=False'

In [27]:
with open('test_trials.json', 'w') as f:
    json.dump(list(random_subset), f)